In [1]:
import pandas as pd
from statsmodels.tsa.ardl import ardl_select_order
from statsmodels.tsa.ardl import ARDL

import numpy as np

import sys
sys.path.append("../src/model")

from ARDL_model import ARDL_model_func

DATA_PATH = "../data/CLEAN"

# Violent movie data

In [2]:
df_box_offices = pd.read_csv(DATA_PATH+"/Violent_Movies_final.tsv", sep="\t")

In [137]:
# Testing out the ARDL model: 
weekly_revenue_films = ARDL_model_func(df_box_offices)

In [138]:
# QUESTIONS
# 1) Only take the years 1950-2012 ? In other years we have nearly no weekly values ? Due to wars ? 
# 2) Fill all missing weeks with 0 films and 0 box_office_revenue


weekly_revenue_films.head()

,Year,Week,no. films released,Box office revenue
0,1913,33,1,980000.0
1,1914,46,1,87028.0
2,1915,6,1,50000000.0
3,1923,47,1,4168790.0
4,1924,49,1,274827.0


# Real world violence data

In [3]:
df_real_violence = pd.read_csv(DATA_PATH+"/FBI_91_12/Scores/0_violence_scores_merged.csv", sep=",")
df_real_violence.head()

,Year,Week,Violence_score
0,1991,1,3798
1,1991,2,2869
2,1991,3,2842
3,1991,4,2871
4,1991,5,3302


In [140]:
# sum up violence counts for all states (grouped by year and week) to have one final violence score for USA per week
weekly_violence_USA = df_real_violence.groupby(["Year", "Week"])["Violence_score"].sum().reset_index()
weekly_violence_USA = weekly_violence_USA.sort_values(["Year", "Week"], ascending=True)
weekly_violence_USA.head()

,Year,Week,Violence_score
0,1991,1,9679
1,1991,2,6439
2,1991,3,6556
3,1991,4,6651
4,1991,5,7336


# Match both datasets in timespan

In [141]:
# we only have the real violence data from 1991 to 2012
year_start = weekly_violence_USA["Year"].min()
year_stop = weekly_violence_USA["Year"].max()

# for 2012, we only have the movie box office revenue until week 42 (included)
df_temp = weekly_revenue_films[weekly_revenue_films['Year'] == 2012]

# Get the maximum value of the "week" column
week_stop_2012 = df_temp['Week'].max()

In [142]:
# cut movie box office revenue dataframe for the timespan 1991-2012
weekly_revenue_films_cut = weekly_revenue_films[(weekly_revenue_films["Year"] >= year_start) & (weekly_revenue_films["Year"] <= year_stop)]

# cut the real world violence dataframe to only contain values until week 42 of 2012 (included)
weekly_violence_USA_cut = weekly_violence_USA[(weekly_violence_USA["Year"] < 2012) | ((weekly_violence_USA["Year"] == 2012) & (weekly_violence_USA["Week"] <= 42))]


# Merge the two dataframes

In [143]:
merged_violence = pd.merge(weekly_violence_USA_cut, weekly_revenue_films_cut, on=['Year', 'Week'], how='left')
merged_violence["no. films released"] = merged_violence["no. films released"].fillna(0).astype(int)
merged_violence["Box office revenue"] = merged_violence["Box office revenue"].fillna(0).astype(int)

merged_violence = merged_violence.sort_values(["Year", "Week"], ascending=True)

merged_violence.head()

,Year,Week,Violence_score,no. films released,Box office revenue
0,1991,1,9679,0,0
1,1991,2,6439,2,38867309
2,1991,3,6556,2,20038851
3,1991,4,6651,0,0
4,1991,5,7336,2,8614328


# Add the time dummies

In [155]:
# Get indicator variables for the year-week
merged_violence["Year-Week"] = merged_violence["Year"].astype(str) + "-" + merged_violence["Week"].astype(str)

# Get indicator variables for year-biweek (to reduce the number of regressors)
merged_violence["BiWeek"] = ((merged_violence["Week"] - 1) // 2) + 1

# Combine year and biweekly group into a "Year-BiWeek" identifier
merged_violence["Year-BiWeek"] = merged_violence["Year"].astype(str) + "-B" + merged_violence["BiWeek"].astype(str)

merged_violence

,Year,Week,Violence_score,no. films released,Box office revenue,Year-Week,BiWeek,Year-BiWeek
0,1991,1,9679,0,0,1991-1,1,1991-B1
1,1991,2,6439,2,38867309,1991-2,1,1991-B1
2,1991,3,6556,2,20038851,1991-3,2,1991-B2
3,1991,4,6651,0,0,1991-4,2,1991-B2
4,1991,5,7336,2,8614328,1991-5,3,1991-B3
...,...,...,...,...,...,...,...,...
1137,2012,38,60275,1,10473039,2012-38,19,2012-B19
1138,2012,39,60948,1,136513833,2012-39,20,2012-B20
1139,2012,40,60768,0,0,2012-40,20,2012-B20
1140,2012,41,56910,1,2005099,2012-41,21,2012-B21


In [177]:
# Create time dummies for biweekly time-fixed effects, drop temporary columns afterwards
time_dummies = pd.get_dummies(merged_violence["Year-BiWeek"], drop_first=True).astype(int)
merged_violence_with_dummies = pd.concat([merged_violence.drop(columns=["Year-Week", "BiWeek", "Year-BiWeek"]), time_dummies], axis=1)

merged_violence_with_dummies.head()

,Year,Week,Violence_score,no. films released,Box office revenue,1991-B10,1991-B11,1991-B12,1991-B13,1991-B14,...,2012-B2,2012-B20,2012-B21,2012-B3,2012-B4,2012-B5,2012-B6,2012-B7,2012-B8,2012-B9
0,1991,1,9679,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1991,2,6439,2,38867309,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1991,3,6556,2,20038851,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1991,4,6651,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1991,5,7336,2,8614328,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [158]:
EXOG = merged_violence_with_dummies.drop(columns=["Year", "Week", "Violence_score", "no. films released"])
EXOG

,Box office revenue,1991-B10,1991-B11,1991-B12,1991-B13,1991-B14,1991-B15,1991-B16,1991-B17,1991-B18,...,2012-B2,2012-B20,2012-B21,2012-B3,2012-B4,2012-B5,2012-B6,2012-B7,2012-B8,2012-B9
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,38867309,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,20038851,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8614328,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1137,10473039,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1138,136513833,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1139,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1140,2005099,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


# Try out ARDL model - Without biweekly time-fixed effects

In [169]:
# Setting the time frame for the auto-regressive part
max_auto_lag = 6            # take into account max. 6 previous timesteps

# Setting the time span for the distributed lag part
max_film_lag = 6            # take into account max. 4 previous timesteps
#max_unemployment_lag = 1    # take into account max 1 previous timestep

In [170]:
# find best order for lags
selected_order = ardl_select_order(
    endog=merged_violence_with_dummies['Violence_score'], 
    exog=EXOG, 
    maxlag=max_auto_lag, 
    maxorder={"Box office revenue": max_film_lag}, 
    ic='aic'
)

/Users/raphael_hellmann/anaconda3/envs/alligators/lib/python3.12/site-packages/statsmodels/tsa/ardl/model.py:455: SpecificationWarning: exog contains variables that are missing from the order dictionary.  Missing keys: 2009-B19, 1996-B2, 2006-B14, 1995-B5, 1998-B3, 1997-B20, 1995-B19, 2007-B2, 2001-B15, 1991-B18, 2006-B2, 2007-B7, 2006-B24, 2008-B3, 1991-B21, 2008-B14, 2004-B10, 2010-B4, 2004-B24, 2006-B21, 2003-B4, 2012-B7, 2007-B17, 1992-B24, 1995-B25, 2001-B8, 1993-B12, 2008-B18, 2011-B2, 2005-B25, 2011-B10, 1995-B14, 1992-B12, 1992-B14, 2009-B21, 2005-B5, 2001-B9, 2007-B13, 2002-B14, 2005-B22, 1996-B23, 1994-B26, 2001-B18, 2010-B2, 2000-B15, 2012-B13, 2004-B27, 1999-B16, 2003-B15, 1998-B2, 1992-B7, 2003-B8, 1999-B8, 2000-B23, 1995-B20, 2003-B16, 1993-B11, 2006-B13, 1995-B9, 1992-B27, 1999-B11, 2004-B21, 1993-B23, 2000-B24, 2011-B23, 1997-B14, 2009-B23, 2012-B11, 2005-B15, 2005-B21, 1993-B20, 2002-B23, 1995-B18, 2009-B15, 2003-B7, 2000-B25, 2004-B26, 2001-B16, 1997-B19, 2010-B20, 20

In [171]:
print(selected_order.ar_lags)
print(selected_order.dl_lags)

[1, 2, 3, 4]
{'Box office revenue': [0, 1, 2, 3, 4]}


In [178]:
ENDOG = merged_violence_with_dummies['Violence_score']

In [180]:
model = ARDL(
    endog=ENDOG,
    exog=EXOG,
    lags=selected_order.ar_lags,
    order=selected_order.dl_lags,
    trend="ct"
).fit()

# Display model summary
print(model.summary())

# Include unemployment rate?

                              ARDL Model Results                              
Dep. Variable:         Violence_score   No. Observations:                 1142
Model:                     ARDL(4, 4)   Log Likelihood              -10546.136
Method:               Conditional MLE   S.D. of innovations           2561.417
Date:                Thu, 12 Dec 2024   AIC                          21116.271
Time:                        16:16:22   BIC                          21176.716
Sample:                             4   HQIC                         21139.100
                                 1142                                         
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   281.2043    163.127      1.724      0.085     -38.863     601.271
trend                     3.7009      0.853      4.336      0.000       2.026       5.375
Violence

/Users/raphael_hellmann/anaconda3/envs/alligators/lib/python3.12/site-packages/statsmodels/tsa/ardl/model.py:455: SpecificationWarning: exog contains variables that are missing from the order dictionary.  Missing keys: 2009-B19, 1996-B2, 2006-B14, 1995-B5, 1998-B3, 1997-B20, 1995-B19, 2007-B2, 2001-B15, 1991-B18, 2006-B2, 2007-B7, 2006-B24, 2008-B3, 1991-B21, 2008-B14, 2004-B10, 2010-B4, 2004-B24, 2006-B21, 2003-B4, 2012-B7, 2007-B17, 1992-B24, 1995-B25, 2001-B8, 1993-B12, 2008-B18, 2011-B2, 2005-B25, 2011-B10, 1995-B14, 1992-B12, 1992-B14, 2009-B21, 2005-B5, 2001-B9, 2007-B13, 2002-B14, 2005-B22, 1996-B23, 1994-B26, 2001-B18, 2010-B2, 2000-B15, 2012-B13, 2004-B27, 1999-B16, 2003-B15, 1998-B2, 1992-B7, 2003-B8, 1999-B8, 2000-B23, 1995-B20, 2003-B16, 1993-B11, 2006-B13, 1995-B9, 1992-B27, 1999-B11, 2004-B21, 1993-B23, 2000-B24, 2011-B23, 1997-B14, 2009-B23, 2012-B11, 2005-B15, 2005-B21, 1993-B20, 2002-B23, 1995-B18, 2009-B15, 2003-B7, 2000-B25, 2004-B26, 2001-B16, 1997-B19, 2010-B20, 20

# Try out ARDL model - With biweekly time-fixed effects

In [159]:
# Setting the time frame for the auto-regressive part
max_auto_lag = 6            # take into account max. 6 previous timesteps

# Setting the time span for the distributed lag part
max_film_lag = 6            # take into account max. 4 previous timesteps
#max_unemployment_lag = 1    # take into account max 1 previous timestep

In [160]:
# find best order for lags
selected_order = ardl_select_order(
    endog=merged_violence_with_dummies['Violence_score'], 
    exog=EXOG, 
    maxlag=max_auto_lag, 
    maxorder={"Box office revenue": max_film_lag}, 
    ic='aic'
)

/Users/raphael_hellmann/anaconda3/envs/alligators/lib/python3.12/site-packages/statsmodels/tsa/ardl/model.py:455: SpecificationWarning: exog contains variables that are missing from the order dictionary.  Missing keys: 2009-B19, 1996-B2, 2006-B14, 1995-B5, 1998-B3, 1997-B20, 1995-B19, 2007-B2, 2001-B15, 1991-B18, 2006-B2, 2007-B7, 2006-B24, 2008-B3, 1991-B21, 2008-B14, 2004-B10, 2010-B4, 2004-B24, 2006-B21, 2003-B4, 2012-B7, 2007-B17, 1992-B24, 1995-B25, 2001-B8, 1993-B12, 2008-B18, 2011-B2, 2005-B25, 2011-B10, 1995-B14, 1992-B12, 1992-B14, 2009-B21, 2005-B5, 2001-B9, 2007-B13, 2002-B14, 2005-B22, 1996-B23, 1994-B26, 2001-B18, 2010-B2, 2000-B15, 2012-B13, 2004-B27, 1999-B16, 2003-B15, 1998-B2, 1992-B7, 2003-B8, 1999-B8, 2000-B23, 1995-B20, 2003-B16, 1993-B11, 2006-B13, 1995-B9, 1992-B27, 1999-B11, 2004-B21, 1993-B23, 2000-B24, 2011-B23, 1997-B14, 2009-B23, 2012-B11, 2005-B15, 2005-B21, 1993-B20, 2002-B23, 1995-B18, 2009-B15, 2003-B7, 2000-B25, 2004-B26, 2001-B16, 1997-B19, 2010-B20, 20

In [ ]:
print(selected_order.ar_lags)
print(selected_order.dl_lags)

[1, 2, 3, 4]


In [ ]:
# Extracting the biweekly time-fixed effects
#time_fixed_effects = EXOG.drop(columns="Box office revenue")

,1991-B10,1991-B11,1991-B12,1991-B13,1991-B14,1991-B15,1991-B16,1991-B17,1991-B18,1991-B19,...,2012-B2,2012-B20,2012-B21,2012-B3,2012-B4,2012-B5,2012-B6,2012-B7,2012-B8,2012-B9
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [176]:
model = ARDL(
    endog=merged_violence_with_dummies['Violence_score'],
    exog=EXOG,
    lags=selected_order.ar_lags,
    order=selected_order.dl_lags,
    fixed=time_dummies,
    trend="ct"
).fit()

# Display model summary
print(model.summary())

# Include unemployment rate?

/Users/raphael_hellmann/anaconda3/envs/alligators/lib/python3.12/site-packages/statsmodels/tsa/ardl/model.py:455: SpecificationWarning: exog contains variables that are missing from the order dictionary.  Missing keys: 2009-B19, 1996-B2, 2006-B14, 1995-B5, 1998-B3, 1997-B20, 1995-B19, 2007-B2, 2001-B15, 1991-B18, 2006-B2, 2007-B7, 2006-B24, 2008-B3, 1991-B21, 2008-B14, 2004-B10, 2010-B4, 2004-B24, 2006-B21, 2003-B4, 2012-B7, 2007-B17, 1992-B24, 1995-B25, 2001-B8, 1993-B12, 2008-B18, 2011-B2, 2005-B25, 2011-B10, 1995-B14, 1992-B12, 1992-B14, 2009-B21, 2005-B5, 2001-B9, 2007-B13, 2002-B14, 2005-B22, 1996-B23, 1994-B26, 2001-B18, 2010-B2, 2000-B15, 2012-B13, 2004-B27, 1999-B16, 2003-B15, 1998-B2, 1992-B7, 2003-B8, 1999-B8, 2000-B23, 1995-B20, 2003-B16, 1993-B11, 2006-B13, 1995-B9, 1992-B27, 1999-B11, 2004-B21, 1993-B23, 2000-B24, 2011-B23, 1997-B14, 2009-B23, 2012-B11, 2005-B15, 2005-B21, 1993-B20, 2002-B23, 1995-B18, 2009-B15, 2003-B7, 2000-B25, 2004-B26, 2001-B16, 1997-B19, 2010-B20, 20

                              ARDL Model Results                              
Dep. Variable:         Violence_score   No. Observations:                 1142
Model:                     ARDL(4, 4)   Log Likelihood               -9503.019
Method:               Conditional MLE   S.D. of innovations           1024.227
Date:                Thu, 12 Dec 2024   AIC                          20178.038
Time:                        15:50:32   BIC                          23129.736
Sample:                             4   HQIC                         21292.828
                                 1142                                         
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   5.45e+04   5.04e+04      1.081      0.280   -4.45e+04    1.54e+05
trend                    10.4441     88.240      0.118      0.906    -162.882     183.771
Violence

In [12]:
test_model = ARDL_model_func(df_box_offices, df_real_violence, time_fixed_effects=False)

/Users/raphael_hellmann/anaconda3/envs/alligators/lib/python3.12/site-packages/statsmodels/tsa/ardl/model.py:455: SpecificationWarning: exog contains variables that are missing from the order dictionary.  Missing keys: 2001-B5, 2005-B21, 2003-B8, 2012-B17, 2005-B25, 2004-B1, 1994-B20, 1994-B3, 2005-B16, 2007-B25, 2011-B22, 2003-B4, 2001-B26, 1993-B14, 2007-B7, 1991-B17, 1995-B21, 1997-B18, 1993-B17, 1992-B23, 2008-B18, 2005-B5, 1997-B20, 1997-B2, 1997-B23, 1997-B16, 1998-B22, 2004-B27, 1991-B24, 2007-B24, 1992-B16, 1996-B11, 2000-B6, 1996-B12, 1993-B7, 1995-B19, 1991-B22, 2002-B13, 2003-B14, 2003-B18, 1998-B21, 2005-B24, 2002-B17, 2006-B12, 2010-B9, 2003-B3, 1999-B21, 1996-B19, 1998-B25, 2003-B5, 1994-B9, 1996-B4, 1992-B17, 1999-B18, 2006-B2, 1992-B1, 2001-B12, 2009-B23, 1995-B8, 2012-B5, 2006-B7, 1995-B15, 1991-B3, 1994-B8, 2010-B3, 1995-B12, 2008-B3, 2004-B10, 2003-B2, 1999-B17, 1996-B14, 1999-B13, 2002-B10, 2004-B25, 2005-B7, 2002-B14, 2009-B26, 1995-B7, 2001-B21, 2000-B8, 1999-B7, 

In [13]:
test_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              ARDL Model Results                              
==============================================================================
Dep. Variable:         Violence_score   No. Observations:                 1142
Model:                     ARDL(4, 4)   Log Likelihood              -10546.136
Method:               Conditional MLE   S.D. of innovations           2561.417
Date:                Thu, 12 Dec 2024   AIC                          21116.271
Time:                        16:48:16   BIC                          21176.716
Sample:                             4   HQIC                         21139.100
                                 1142                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   281.2043    163.127      1.724      0.085     -38.863     601.271
trend                     3.7009      0.853      4.336      0.000       2.026       5.375
Violence_score.L1         0.5672      0.029     19.292      0.000       0.510       0.625
Violence_score.L2         0.1239      0.034      3.679      0.000       0.058       0.190
Violence_score.L3         0.0922      0.034      2.737      0.006       0.026       0.158
Violence_score.L4         0.1515      0.029      5.159      0.000       0.094       0.209
Box office revenue.L0  1.189e-07   3.11e-07      0.382      0.703   -4.92e-07     7.3e-07
Box office revenue.L1 -1.006e-07   3.11e-07     -0.324      0.746   -7.11e-07    5.09e-07
Box office revenue.L2 -3.937e-07   3.11e-07     -1.267      0.206      -1e-06    2.16e-07
Box office revenue.L3 -7.849e-07   3.11e-07     -2.521      0.012    -1.4e-06   -1.74e-07
Box office revenue.L4  6.056e-07   3.12e-07      1.939      0.053   -7.35e-09    1.22e-06
=========================================================================================
"""